## librerias

In [41]:
import pandas as pd
import numpy as np
import os
import matplotlib as plt
import seaborn as sns
import scipy.interpolate as interpolate

## Carga de datos 

In [42]:
# Leer el archivo CSV
data_latinoamerica = pd.read_csv("data_latinoamerica.csv")

## Se crea un grupo de paises sobre los cuales se estudiara los efectos y concecuencias del COVId-19

In [43]:
#creo una lista con los paises seleccionados:
Paises_seleccionados = ["Colombia", "Argentina", "Chile", "Mexico", "Peru" , "Brazil"]
#se crea un amascara por paises y 
data_latinoamerica = data_latinoamerica[data_latinoamerica["country_name"].isin(Paises_seleccionados)]

## Se procede a filtar el dataset para crear el estudio desde el 01/01/2021

In [44]:
# filtramos los datos por fechas para trabajar con datos posteriores al 31/12/2020

data_latinoamerica = data_latinoamerica [data_latinoamerica["date"] > "2020-12-31" ]

data_latinoamerica.shape

(7549375, 50)

In [45]:
data_latinoamerica.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7549375 entries, 366 to 12136776
Data columns (total 50 columns):
 #   Column                                 Dtype  
---  ------                                 -----  
 0   location_key                           object 
 1   date                                   object 
 2   country_code                           object 
 3   country_name                           object 
 4   new_confirmed                          float64
 5   new_deceased                           float64
 6   cumulative_confirmed                   float64
 7   cumulative_deceased                    float64
 8   cumulative_vaccine_doses_administered  float64
 9   population                             float64
 10  population_male                        float64
 11  population_female                      float64
 12  population_rural                       float64
 13  population_urban                       float64
 14  population_density                     float64
 15  

## Contamos la cantidad de nulos de todas las columnas 

In [46]:
# Contar los valores nulos en cada columna
valores_nulos = data_latinoamerica.isnull().sum()
print("Cantidad de valores nulos por columna:")
print(valores_nulos)

Cantidad de valores nulos por columna:
location_key                                   0
date                                           0
country_code                                   0
country_name                                   0
new_confirmed                            3164800
new_deceased                             3167470
cumulative_confirmed                     3164800
cumulative_deceased                      3167470
cumulative_vaccine_doses_administered    7286122
population                                328750
population_male                           708125
population_female                         708125
population_rural                         7545625
population_urban                         7545625
population_density                       7545625
human_development_index                  4081875
population_age_00_09                      924375
population_age_10_19                      924375
population_age_20_29                      924375
population_age_30_39          

## Contamos la cantidad de elementos que tiene cada una de las columnas

In [47]:
# Contar los valores no nulos en cada columna
valores_no_nulos = data_latinoamerica.notnull().sum()
print("Cantidad de valores no nulos por columna:")
print(valores_no_nulos)


Cantidad de valores no nulos por columna:
location_key                             7549375
date                                     7549375
country_code                             7549375
country_name                             7549375
new_confirmed                            4384575
new_deceased                             4381905
cumulative_confirmed                     4384575
cumulative_deceased                      4381905
cumulative_vaccine_doses_administered     263253
population                               7220625
population_male                          6841250
population_female                        6841250
population_rural                            3750
population_urban                            3750
population_density                          3750
human_development_index                  3467500
population_age_00_09                     6625000
population_age_10_19                     6625000
population_age_20_29                     6625000
population_age_30_39       

# se verifica la que tipo de elemento tienen las columnas 'location_key', 'country_code' y 'country_name'

In [48]:
columnas_relevantes = ['location_key', 'country_code', 'country_name']
datos_unicos = data_latinoamerica[columnas_relevantes].drop_duplicates()

print(datos_unicos)


         location_key country_code country_name
366                AR           AR    Argentina
1357             AR_A           AR    Argentina
2348         AR_A_007           AR    Argentina
3339         AR_A_014           AR    Argentina
4330         AR_A_021           AR    Argentina
...               ...          ...          ...
12132188  PE_UCA_0302           PE         Peru
12133179  PE_UCA_0303           PE         Peru
12134170  PE_UCA_0304           PE         Peru
12135161  PE_UCA_0305           PE         Peru
12136152  PE_UCA_0401           PE         Peru

[12079 rows x 3 columns]


### para poder entender mejor los valores dentro de 'location_key' se realiza un conteo de los valores unicos de esta columna

In [49]:
data_latinoamerica["location_key"].value_counts()

location_key
PE_UCA_0401    625
AR             625
AR_A           625
AR_A_007       625
AR_A_014       625
              ... 
AR_A_147       625
AR_A_140       625
AR_A_133       625
AR_A_126       625
AR_A_119       625
Name: count, Length: 12079, dtype: int64

### podemos apreciar la cantidad de valores repetidos entro de estas filas, por esta razon filtramos la columna 'location_key' por los siguientes valores "AR", "BR", "CO", "CL", "MX", "PE", que son los valores por país dejando de lado el analisis perticular de cada region dentro de estos. 

In [50]:
# Primer filtro: quedarse solo con los países de interés
data_latinoamerica = data_latinoamerica[data_latinoamerica["country_name"].isin(Paises_seleccionados)]

# Segundo filtro: quedarse solo con los datos agregados a nivel nacional
data_latinoamerica = data_latinoamerica[data_latinoamerica["location_key"].isin(["AR", "BR", "CO", "CL", "MX", "PE"])]


como las columnas 'location_key' y 'country_code' no aportan informacion relevante, para el analisis general de cada pais, estas seran eliminadas

In [51]:
# eliminamos las columnas 'location_key' y 'country_code'
data_latinoamerica = data_latinoamerica.drop(columns=['location_key', 'country_code'])

## Se completan los valores faltantes (null) de las columnas 'new_recovered', 'new_deceased' y 'new_confirmed' con ceros


In [52]:
# Rellenar los valores nulos en las columnas 'new_recovered', 'new_deceased' y 'new_confirmed' con ceros
data_latinoamerica['new_recovered'] = data_latinoamerica['new_recovered'].fillna(0)
data_latinoamerica['new_deceased'] = data_latinoamerica['new_deceased'].fillna(0)
data_latinoamerica['new_confirmed'] = data_latinoamerica['new_confirmed'].fillna(0)



## verificamos que se realizen los cambios sin alterr el dataframe y se verifican nulos en otras columnas

In [53]:
data_latinoamerica.isna().sum()

date                                        0
country_name                                0
new_confirmed                               0
new_deceased                                0
cumulative_confirmed                       21
cumulative_deceased                        21
cumulative_vaccine_doses_administered     589
population                                  0
population_male                             0
population_female                           0
population_rural                            0
population_urban                            0
population_density                          0
human_development_index                     0
population_age_00_09                        0
population_age_10_19                        0
population_age_20_29                        0
population_age_30_39                        0
population_age_40_49                        0
population_age_50_59                        0
population_age_60_69                        0
population_age_70_79              

## Completamos la columna de new_recovered con el promedio de cada pais.

In [54]:
# Calcula el promedio de 'new_recovered' por país
promedio_recuperados_por_pais = data_latinoamerica.groupby('country_name')['new_recovered'].mean()

# Rellena los valores nulos en 'new_recovered' con el promedio específico para cada país
for pais in data_latinoamerica['country_name'].unique():
    mask = (data_latinoamerica['country_name'] == pais)
    data_latinoamerica.loc[mask, 'new_recovered'].fillna(promedio_recuperados_por_pais[pais], inplace=True)

# Verifica si los valores nulos se han llenado correctamente
nulos_new_recovered = data_latinoamerica['new_recovered'].isnull().sum()
print("Cantidad de valores nulos en 'new_recovered' después de llenarlos:", nulos_new_recovered)


Cantidad de valores nulos en 'new_recovered' después de llenarlos: 0


## En el caso de las columnas 'cumulative_confirmed','cumulative_deceased' y 'cumulative_recovered' completamos considerando que el primer dato de cada pais es nulo por lo que se asigna el primer dato first_row [] de cada uno, y en segundo lugar se completa con el acumulado anterior utilizando cumsum()

In [55]:
# Verificación del primer dato
for pais in data_latinoamerica['country_name'].unique():
    mask = (data_latinoamerica['country_name'] == pais)
    first_row = data_latinoamerica.loc[mask].iloc[0]
    
    if pd.isnull(first_row['cumulative_confirmed']):
        data_latinoamerica.loc[mask, 'cumulative_confirmed'] = first_row['new_confirmed']
    
    if pd.isnull(first_row['cumulative_deceased']):
        data_latinoamerica.loc[mask, 'cumulative_deceased'] = first_row['new_deceased']
    
    if pd.isnull(first_row['cumulative_recovered']):
        data_latinoamerica.loc[mask, 'cumulative_recovered'] = first_row['new_recovered']

# Cálculo acumulativo para filas restantes
for pais in data_latinoamerica['country_name'].unique():
    mask = (data_latinoamerica['country_name'] == pais)
    data_latinoamerica.loc[mask, 'cumulative_confirmed'] = data_latinoamerica.loc[mask, 'new_confirmed'].cumsum()
    data_latinoamerica.loc[mask, 'cumulative_deceased'] = data_latinoamerica.loc[mask, 'new_deceased'].cumsum()
    data_latinoamerica.loc[mask, 'cumulative_recovered'] = data_latinoamerica.loc[mask, 'new_recovered'].cumsum()

# Verificación de valores nulos después de completar los datos
nulos_despues_de_eliminar = data_latinoamerica.isnull().sum()
print("Cantidad de valores nulos después de completar los datos:")
print(nulos_despues_de_eliminar)


Cantidad de valores nulos después de completar los datos:
date                                       0
country_name                               0
new_confirmed                              0
new_deceased                               0
cumulative_confirmed                       0
cumulative_deceased                        0
cumulative_vaccine_doses_administered    589
population                                 0
population_male                            0
population_female                          0
population_rural                           0
population_urban                           0
population_density                         0
human_development_index                    0
population_age_00_09                       0
population_age_10_19                       0
population_age_20_29                       0
population_age_30_39                       0
population_age_40_49                       0
population_age_50_59                       0
population_age_60_69                      

## Completamos la columna 'cumulative_vaccine_doses_administered'; en este caso se toma el dato anterior y, en el caso de que fuera el primer dato del subgrupo se toma el dato posterior. Des esta forma se da continuidad a la curva en la grafica.

In [56]:
# Ordenamos los datos previo a completar los acumulados de las vacunaciones
data_latinoamerica = data_latinoamerica.sort_values(by=['country_name', 'date'], ascending=True)

In [57]:
# Reemplazar los valores nulos con el valor siguiente (forward fill)
data_latinoamerica['cumulative_vaccine_doses_administered'] = data_latinoamerica.groupby('country_name')['cumulative_vaccine_doses_administered'].ffill()

# Reemplazar los valores nulos restantes con el valor anterior (backward fill)
data_latinoamerica['cumulative_vaccine_doses_administered'] = data_latinoamerica.groupby('country_name')['cumulative_vaccine_doses_administered'].bfill()


In [58]:
# Verificación de valores nulos después de completar los datos
nulos_despues_de_eliminar = data_latinoamerica.isnull().sum()
print("Cantidad de valores nulos después de completar los datos:")
print(nulos_despues_de_eliminar)

Cantidad de valores nulos después de completar los datos:
date                                      0
country_name                              0
new_confirmed                             0
new_deceased                              0
cumulative_confirmed                      0
cumulative_deceased                       0
cumulative_vaccine_doses_administered     0
population                                0
population_male                           0
population_female                         0
population_rural                          0
population_urban                          0
population_density                        0
human_development_index                   0
population_age_00_09                      0
population_age_10_19                      0
population_age_20_29                      0
population_age_30_39                      0
population_age_40_49                      0
population_age_50_59                      0
population_age_60_69                      0
population_age_70_

In [59]:
data_latinoamerica.head()

,date,country_name,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,cumulative_vaccine_doses_administered,population,population_male,population_female,...,population_largest_city,area_rural_sq_km,area_urban_sq_km,life_expectancy,adult_male_mortality_rate,adult_female_mortality_rate,pollution_mortality_rate,comorbidity_mortality_rate,new_recovered,cumulative_recovered
366,2021-01-01,Argentina,2685.0,140.0,2685.0,140.0,18849.0,44938712.0,19523766.0,20593330.0,...,15057273.0,2690269.0,55032.0,76.52,147.086,79.483,26.6,15.8,0.0,0.0
367,2021-01-02,Argentina,7767.0,166.0,10452.0,306.0,20334.0,44938712.0,19523766.0,20593330.0,...,15057273.0,2690269.0,55032.0,76.52,147.086,79.483,26.6,15.8,0.0,0.0
368,2021-01-03,Argentina,4934.0,157.0,15386.0,463.0,20529.0,44938712.0,19523766.0,20593330.0,...,15057273.0,2690269.0,55032.0,76.52,147.086,79.483,26.6,15.8,0.0,0.0
369,2021-01-04,Argentina,13953.0,157.0,29339.0,620.0,25164.0,44938712.0,19523766.0,20593330.0,...,15057273.0,2690269.0,55032.0,76.52,147.086,79.483,26.6,15.8,0.0,0.0
370,2021-01-05,Argentina,14085.0,160.0,43424.0,780.0,29852.0,44938712.0,19523766.0,20593330.0,...,15057273.0,2690269.0,55032.0,76.52,147.086,79.483,26.6,15.8,0.0,0.0


## Guardamos el archivo con un grado elevado de limpieza y transformacion para la siguiente etapa EDA

In [60]:
# Guardamos el dataset ya transformado
data_latinoamerica.to_csv('data_latinoamerica_clean.csv', index=False)